In [7]:
import pymongo 
from pymongo import MongoClient


In [8]:
# connect to database
connection = MongoClient('localhost', 27017)

db = connection.sample


In [10]:
# handle to names collection
test = db.test

item = test.insert_one( { "_id" : 'one', "value": 1 } )


In [13]:
print item.acknowledged
print item.inserted_id

True
one


In [15]:
test.find_one()

{u'_id': u'one', u'value': 1}

In [ ]:
for post in posts.find():